In [ ]:
import sklearn

In [ ]:
print(sklearn.__version__)

In [ ]:
pip install scikit-learn==0.21.2

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files += [os.path.join(dirname, filename)]
print(files)

In [ ]:
df = pd.read_csv(files[0])
df

### We don't need the columns RowNumber, CustomerId, Surname for Classification. So, let's drop them!

In [ ]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace=True) 
df

In [ ]:
df.info()

#### There are no NaNs.

## Let's handle the categorical variables 

In [ ]:
df = pd.get_dummies(df, drop_first=True)
df

## Train-Test Split

In [ ]:
y = df.Exited
X = df.drop(['Exited'], axis=1)
X

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

## Let's perform feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
test_x = sc.transform(test_x)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
# from keras.layers import LeakyReLU, PReLU, ELU
from keras.layers import Dropout

In [ ]:
classifier = Sequential()
# Input Dim and First Hidden layer
classifier.add(Dense(units=6, kernel_initializer='he_uniform', activation='relu', input_dim=11))
# Second Hidden layer
classifier.add(Dense(units=6, kernel_initializer='he_uniform', activation='relu'))
# Output layer with Sigmoid Activation
classifier.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))

In [ ]:
classifier.summary()

In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_history = classifier.fit(train_x, train_y, validation_split=0.33, batch_size=10, epochs=100)

In [ ]:
print(model_history.history.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_loss'])
plt.plot(model_history.history['val_accuracy'])
plt.legend(model_history.history.keys())
plt.show()

In [ ]:
pred = classifier.predict(test_x)
pred = (pred > 0.5)
pred

## Model Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, pred)
cm

In [ ]:
from sklearn.metrics import f1_score
f1_score = f1_score(test_y, pred)
print(f1_score)

In [ ]:
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(test_y, pred)
acc_score

## HyperParameter Tuning

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization
from keras.activations import relu, sigmoid

In [ ]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim=train_x.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=2)

In [ ]:
layers = ([20], [40,20], [45,30,15])
activations = ('sigmoid', 'relu')
param_grid = dict(layers=layers, activation=activations, batch_size=(128,256), epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

In [ ]:
param_grid

In [ ]:
%%time
grid_result = grid.fit(train_x, train_y)

In [ ]:
print(grid_result.best_score_, grid_result.best_params_)

In [ ]:
print(grid_result.history.keys())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(grid_result.history['loss'])
plt.plot(grid_result.history['accuracy'])
plt.plot(grid_result.history['val_loss'])
plt.plot(grid_result.history['val_accuracy'])
plt.legend(grid_result.history.keys())
plt.show()

In [ ]:
pred = grid.predict(test_x)
pred = (pred > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, pred)
print('Confusion Matrix:\n', cm)

from sklearn.metrics import f1_score
f1_score = f1_score(test_y, pred)
print('F1 Score: ',f1_score)

from sklearn.metrics import accuracy_score
acc_score = accuracy_score(test_y, pred)
print('Accuracy Score: ',acc_score)